In [28]:
from pyspark.sql import SparkSession, Row
from pyspark.ml.feature import VectorAssembler
import numpy as np
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from dataclasses import dataclass
from pyspark.ml.linalg import Vectors


In [2]:
import os
print(os.getcwd())

/Users/blambright/Downloads/Big Data Analytics/wedding-song-recommendations/modeling


In [3]:
# creating spark app
spark = SparkSession.builder.appName("Creating_Clustering_Model").getOrCreate()

#reading in the CSV file 
df = spark.read.csv('numpy_array_for_modeling.csv', header=True, inferSchema=True)

df.printSchema()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/29 14:54:20 WARN Utils: Your hostname, Bens-MacBook-Air-7.local, resolves to a loopback address: 127.0.0.1; using 172.20.10.3 instead (on interface en0)
25/11/29 14:54:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/29 14:54:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/29 14:54:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


root
 |-- rank: double (nullable = true)
 |-- trend: double (nullable = true)
 |-- streams: double (nullable = true)
 |-- popularity: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- af_danceability: double (nullable = true)
 |-- af_energy: double (nullable = true)
 |-- af_key: double (nullable = true)
 |-- af_loudness: double (nullable = true)
 |-- af_mode: double (nullable = true)
 |-- af_speechiness: double (nullable = true)
 |-- af_acousticness: double (nullable = true)
 |-- af_instrumentalness: double (nullable = true)
 |-- af_liveness: double (nullable = true)
 |-- af_valence: double (nullable = true)
 |-- af_tempo: double (nullable = true)
 |-- af_time_signature: double (nullable = true)
 |-- language_id: double (nullable = true)
 |-- date: double (nullable = true)



In [41]:
input_columns =  ["af_danceability","af_energy","af_speechiness","af_acousticness","language_id","date"]
assembler = VectorAssembler(inputCols = input_columns, outputCol = 'features')
vector_df = assembler.transform(df)

collected_vectors = vector_df.select('features').collect()
#printing one vector
print("Collected Vectors")
print(collected_vectors[:1])

Collected Vectors
[Row(features=DenseVector([0.852, 0.773, 0.0776, 0.187, 1.0, 2017.0]))]


In [42]:
numpy_feature_matrix = np.asarray([row[0].toArray() for row in collected_vectors])

print('Numpy Feature Matrix from Vector Column:')
print(numpy_feature_matrix)

Numpy Feature Matrix from Vector Column:
[[8.520e-01 7.730e-01 7.760e-02 1.870e-01 1.000e+00 2.017e+03]
 [6.630e-01 9.200e-01 2.260e-01 4.310e-03 1.000e+00 2.016e+03]
 [7.610e-01 8.380e-01 5.020e-02 4.000e-01 1.000e+00 2.016e+03]
 ...
 [6.800e-01 7.300e-01 3.150e-02 1.000e-01 1.000e+00 2.016e+03]
 [7.230e-01 8.090e-01 6.250e-02 3.460e-03 0.000e+00 2.015e+03]
 [5.340e-01 7.340e-01 2.290e-01 1.830e-02 1.000e+00 2.015e+03]]


In [43]:
vector_df.printSchema()

root
 |-- rank: double (nullable = true)
 |-- trend: double (nullable = true)
 |-- streams: double (nullable = true)
 |-- popularity: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- af_danceability: double (nullable = true)
 |-- af_energy: double (nullable = true)
 |-- af_key: double (nullable = true)
 |-- af_loudness: double (nullable = true)
 |-- af_mode: double (nullable = true)
 |-- af_speechiness: double (nullable = true)
 |-- af_acousticness: double (nullable = true)
 |-- af_instrumentalness: double (nullable = true)
 |-- af_liveness: double (nullable = true)
 |-- af_valence: double (nullable = true)
 |-- af_tempo: double (nullable = true)
 |-- af_time_signature: double (nullable = true)
 |-- language_id: double (nullable = true)
 |-- date: double (nullable = true)
 |-- features: vector (nullable = true)



In [48]:
row = Row(
    rank=1.0, trend=1.0, streams=253019.0, popularity=78.0, duration_ms=195840.0,
    af_danceability=0.852, af_energy=0.773, af_key=8.0, af_loudness=-2.921, af_mode=0.0,
    af_speechiness=0.0776, af_acousticness=0.187, af_instrumentalness=3.05E-5, af_liveness=0.159,
    af_valence=0.907, af_tempo=102.034, af_time_signature=4.0, language_id=0.0, date=2016.0,
    features=Vectors.dense([0.899,0.626,0.292,0.076,0.0,2016.0])
)

input_columns =  ["af_danceability","af_energy","af_speechiness","af_acousticness","language_id","date"]


# Convert Row to Python list
# row_list = row.to_list()
new_row_df = spark.createDataFrame([row])
new_row_df.show()

+----+-----+--------+----------+-----------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+-----------+------+--------------------+
|rank|trend| streams|popularity|duration_ms|af_danceability|af_energy|af_key|af_loudness|af_mode|af_speechiness|af_acousticness|af_instrumentalness|af_liveness|af_valence|af_tempo|af_time_signature|language_id|  date|            features|
+----+-----+--------+----------+-----------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+-----------+------+--------------------+
| 1.0|  1.0|253019.0|      78.0|   195840.0|          0.852|    0.773|   8.0|     -2.921|    0.0|        0.0776|          0.187|            3.05E-5|      0.159|     0.907| 102.034|              4.0|        0.0|2016.0|[0.899,0.626,0.29...|
+----+-----+--------+----------+-----------+

In [49]:
# train the model 
kmeans_model = KMeans(k=2, seed = 42)
model = kmeans_model.fit(vector_df)

# make predictions
prediction = model.transform(new_row_df)
prediction.show()

+----+-----+--------+----------+-----------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+-----------+------+--------------------+----------+
|rank|trend| streams|popularity|duration_ms|af_danceability|af_energy|af_key|af_loudness|af_mode|af_speechiness|af_acousticness|af_instrumentalness|af_liveness|af_valence|af_tempo|af_time_signature|language_id|  date|            features|prediction|
+----+-----+--------+----------+-----------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+-----------+------+--------------------+----------+
| 1.0|  1.0|253019.0|      78.0|   195840.0|          0.852|    0.773|   8.0|     -2.921|    0.0|        0.0776|          0.187|            3.05E-5|      0.159|     0.907| 102.034|              4.0|        0.0|2016.0|[0.899,0.626,0.29...|         1|


In [50]:
vector_df.show(5)

+----+-----+--------+----------+-----------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+-----------+------+--------------------+
|rank|trend| streams|popularity|duration_ms|af_danceability|af_energy|af_key|af_loudness|af_mode|af_speechiness|af_acousticness|af_instrumentalness|af_liveness|af_valence|af_tempo|af_time_signature|language_id|  date|            features|
+----+-----+--------+----------+-----------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+-----------+------+--------------------+
| 1.0|  1.0|253019.0|      78.0|   195840.0|          0.852|    0.773|   8.0|     -2.921|    0.0|        0.0776|          0.187|            3.05E-5|      0.159|     0.907| 102.034|              4.0|        1.0|2017.0|[0.852,0.773,0.07...|
| 2.0|  2.0|223988.0|      72.0|   259195.0|

In [52]:
# Add prediction column to the full dataset
vector_df_with_clusters = model.transform(vector_df)
# vector_df_with_clusters.groupBy("prediction").count().show()
vector_df_with_clusters.filter("prediction = 0").show(5)

# Get the cluster ID for your new row
cluster_id = model.transform(new_row_df).select("prediction").collect()[0][0]

# Filter dataset to rows in the same cluster
similar_rows = vector_df_with_clusters.filter(f"prediction = {cluster_id}")
# similar_rows.show()

+-----+-----+-------+----------+-----------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+-----------+------+--------------------+----------+
| rank|trend|streams|popularity|duration_ms|af_danceability|af_energy|af_key|af_loudness|af_mode|af_speechiness|af_acousticness|af_instrumentalness|af_liveness|af_valence|af_tempo|af_time_signature|language_id|  date|            features|prediction|
+-----+-----+-------+----------+-----------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+-----------+------+--------------------+----------+
|104.0|  1.0|23090.0|       0.0|   182973.0|          0.841|    0.902|   7.0|     -5.114|    1.0|        0.0308|          0.534|            1.53E-4|      0.054|     0.942|  98.044|              4.0|        1.0|1980.0|[0.841,0.902,0.03...|         0|
